In [1]:
import os
import pickle
import json
from tqdm import tqdm

In [2]:
from collections import defaultdict

In [3]:
os.environ["CODE"] = "/home/aarslan/mq"


In [4]:
with open(os.path.join(os.environ["SCRATCH"], "ego4d_data/v2/analysis_data/dependency_parsing_results/dependency_parsing_results.pickle"), "rb") as reader:
    dependency_parsing_results = pickle.load(reader)


In [5]:
with open(os.path.join(os.environ["SCRATCH"], "ego4d_data/v2/analysis_data/ground_truth_labels/ground_truth_labels.pickle"), "rb") as reader:
    ground_truth_labels = pickle.load(reader)


In [6]:
with open(os.path.join(os.environ["CODE"], "scripts/06_analyze_frame_features/03_map_label_dependency_parsing_features_and_blip2_answer_dependency_parsing_features", "label_verb_noun_tool_mapping.json"), "r") as reader:
    label_verb_noun_tools_mapping = json.load(reader)


In [7]:
label_verb_noun_tools_mapping_keys = sorted(list(label_verb_noun_tools_mapping.keys())) + ["background"]

label_index_label_mapping = dict()
for i in range(len(label_verb_noun_tools_mapping_keys)):
    label_index_label_mapping[i] = label_verb_noun_tools_mapping_keys[i]


In [8]:
blip2_question_index_verb_noun_tool_match_count_mapping = defaultdict(lambda: 0)

blip2_question_index_verb_noun_match_count_mapping = defaultdict(lambda: 0)

blip2_question_index_verb_tool_match_count_mapping = defaultdict(lambda: 0)

blip2_question_index_verb_match_count_mapping = defaultdict(lambda: 0)

blip2_question_index_noun_match_count_mapping = defaultdict(lambda: 0)

blip2_question_index_tool_match_count_mapping = defaultdict(lambda: 0)

blip2_question_index_background_match_count_mapping = defaultdict(lambda: 0)

blip2_question_index_count_mapping = defaultdict(lambda: 0)

for clip_id in tqdm(ground_truth_labels.keys()):
    for frame_id in ground_truth_labels[clip_id].keys():
        current_dependency_parsing_results = dependency_parsing_results[clip_id][int((frame_id // 6) * 6)]
        for blip2_question_index, blip2_answer_verb_noun_tool_pairs in current_dependency_parsing_results.items():
            blip2_answer = blip2_answer_verb_noun_tool_pairs[0]
            blip2_verb_noun_tool_pairs = blip2_answer_verb_noun_tool_pairs[1]
            for blip2_verb_noun_tool_pair in blip2_verb_noun_tool_pairs:
                blip2_verb = blip2_verb_noun_tool_pair[0]
                blip2_noun = blip2_verb_noun_tool_pair[1]
                blip2_tool = blip2_verb_noun_tool_pair[2]

                current_ground_truth_label_indices = ground_truth_labels[clip_id][frame_id]
                for current_ground_truth_label_index in current_ground_truth_label_indices:
                    if current_ground_truth_label_index == len(label_verb_noun_tools_mapping_keys) - 1:
                        at_least_one_label_matches = False
                        for label, label_verb_noun_tool_pairs in label_verb_noun_tools_mapping.items():
                            if at_least_one_label_matches:
                                break
                            for label_verb_noun_tool_pair in label_verb_noun_tool_pairs:
                                label_verb = label_verb_noun_tool_pair[0]
                                label_noun = label_verb_noun_tool_pair[1]
                                label_tool = label_verb_noun_tool_pair[2]
                                if blip2_verb == label_verb or blip2_noun == label_noun:
                                    at_least_one_label_matches = True
                                    break
                        if not at_least_one_label_matches:
                            blip2_question_index_background_match_count_mapping[blip2_question_index] += 1
                        blip2_question_index_count_mapping[blip2_question_index] += 1
                    else:
                        current_ground_truth_label = label_verb_noun_tools_mapping_keys[current_ground_truth_label_index]
                        current_ground_truth_label_verb_noun_tool_pairs = label_verb_noun_tools_mapping[current_ground_truth_label]

                        for current_ground_truth_label_verb_noun_tool_pair in current_ground_truth_label_verb_noun_tool_pairs:
                            current_ground_truth_label_verb = current_ground_truth_label_verb_noun_tool_pair[0]
                            current_ground_truth_label_noun = current_ground_truth_label_verb_noun_tool_pair[1]
                            current_ground_truth_label_tool = current_ground_truth_label_verb_noun_tool_pair[2]

                            blip2_question_index_count_mapping[blip2_question_index] += 1

                            # Verb, Noun, Tool match
                            if blip2_verb == current_ground_truth_label_verb and blip2_noun == current_ground_truth_label_noun and blip2_tool == current_ground_truth_label_tool:
                                blip2_question_index_verb_noun_tool_match_count_mapping[blip2_question_index] += 1

                            # Verb, Noun match
                            if blip2_verb == current_ground_truth_label_verb and blip2_noun == current_ground_truth_label_noun:
                                blip2_question_index_verb_noun_match_count_mapping[blip2_question_index] += 1

                            # Verb, Tool match
                            if blip2_verb == current_ground_truth_label_verb and blip2_tool == current_ground_truth_label_tool:
                                blip2_question_index_verb_tool_match_count_mapping[blip2_question_index] += 1

                            # Verb match
                            if blip2_verb == current_ground_truth_label_verb:
                                blip2_question_index_verb_match_count_mapping[blip2_question_index] += 1

                            # Noun match
                            if blip2_noun == current_ground_truth_label_noun:
                                blip2_question_index_noun_match_count_mapping[blip2_question_index] += 1

for mapping in [blip2_question_index_verb_noun_tool_match_count_mapping,
                blip2_question_index_verb_noun_match_count_mapping,
                blip2_question_index_verb_tool_match_count_mapping,
                blip2_question_index_verb_match_count_mapping,
                blip2_question_index_noun_match_count_mapping,
                blip2_question_index_background_match_count_mapping]:
    for blip2_question_index in mapping.keys():
        mapping[blip2_question_index] = mapping[blip2_question_index] / float(blip2_question_index_count_mapping[blip2_question_index])


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 2495/2495 [3:11:12<00:00,  4.60s/it]


In [19]:
blip2_question_index_verb_noun_tool_match_count_mapping[0], blip2_question_index_verb_noun_match_count_mapping[0], blip2_question_index_verb_tool_match_count_mapping[0], blip2_question_index_verb_match_count_mapping[0], blip2_question_index_noun_match_count_mapping[0], blip2_question_index_background_match_count_mapping[0]



(0.001673558175534314,
 0.0024362035019285405,
 0.008458050456956369,
 0.012660901898439667,
 0.036867499051467925,
 0.16677267479626207)

In [20]:
blip2_question_index_verb_noun_tool_match_count_mapping[1], blip2_question_index_verb_noun_match_count_mapping[1], blip2_question_index_verb_tool_match_count_mapping[1], blip2_question_index_verb_match_count_mapping[1], blip2_question_index_noun_match_count_mapping[1], blip2_question_index_background_match_count_mapping[1]



(0.0025997659461370884,
 0.004234213946921597,
 0.028600258537299736,
 0.03528592124451193,
 0.0595242355176737,
 0.12786248393883298)

In [21]:
blip2_question_index_verb_noun_tool_match_count_mapping[2], blip2_question_index_verb_noun_match_count_mapping[2], blip2_question_index_verb_tool_match_count_mapping[2], blip2_question_index_verb_match_count_mapping[2], blip2_question_index_noun_match_count_mapping[2], blip2_question_index_background_match_count_mapping[2]



(0.0024523339497153425,
 0.0045796340834405435,
 0.015427533505265623,
 0.0234195709374107,
 0.05627053734020419,
 0.14063185921021393)